In [ ]:
from openai import OpenAI
import os
import yaml
import json

model = OpenAI(api_key='???')

### Reproduce results from paper repo

In [ ]:
dataset_names = {
    # 'abt-buy-sampled-gs_domain-complex-force': 'abt-buy-gs',
    'walmart-amazon-sampled-gs_domain-complex-force': 'walmart-amazon-gs',
    # 'amazon-google-sampled-gs_domain-complex-force': 'amazon-google-gs',
    # 'dblp-scholar-sampled-gs_domain-complex-force': 'dblp-scholar-gs',
    # 'wdcproducts-80cc-seen-sampled-250-gs-2_domain-complex-force': 'wdcproducts-80cc-seen-gs'
}


for d in dataset_names.keys():
    
    output = []

    # read in data
    with open(f'./LLMForEM/prompt-answer-combined/prompts_and_answers/{d}_default_gpt-4-0613_run-1.jsonl', 'r') as f:
        lines = [json.loads(line) for line in f]
        
    for line in lines:
        
        message = [
            {'role': 'user', 'content': line['prompt']},
        ]
        
        response = model.chat.completions.create(
            model='gpt-4-0613',
            messages=message,
            temperature=0
        )
        ans = response.choices[0].message.content
        
        # add to output
        output.append({
            "prompt": line['prompt'],
            "original_answer": line['answer'],
            "answer": response.choices[0].message.content
        })
    
    with open(f"./no-schema-gpt4/{d}_results.json", "w") as out:
            json.dump(output, out, indent=4)

### Benchmark with schema

In [ ]:
dataset_names = {
    'abt-buy-sampled-gs_domain-complex-force': 'abt-buy-gs',
    # 'walmart-amazon-sampled-gs_domain-complex-force': 'walmart-amazon-gs',
    # 'amazon-google-sampled-gs_domain-complex-force': 'amazon-google-gs',
    # 'dblp-scholar-sampled-gs_domain-complex-force': 'dblp-scholar-gs',
    # 'dblp-acm-sampled-gs_domain-complex-force': 'dblp-acm-gs'
    # 'wdcproducts-80cc-seen-sampled-250-gs-2_domain-complex-force': 'wdcproducts-80cc-seen-gs'
}

for name in dataset_names.values():
    count = 0
    with open(f"./datasets/{name}.json", "r") as f:
        
        output = []
        
        for line in f:
        
            # format data
            mid_split = line.find(',"id_right') # split 2 entities
            right_split = line.find(',"label') # remove label
            left = line[0:mid_split].replace("_left", "") + '}'
            right = line[mid_split+1:right_split].replace("_right", "") + '}'
            label = line[right_split+9:right_split+10]
            
            # query
            message = [
                {'role': 'user', 'content': "Do the two product descriptions refer to the same real-world product? Answer with 'Yes' if they do and 'No' if they do not."},
                {'role': 'user', 'content': "product 1:" + left},
                {'role': 'user', 'content': "product 2:" + right}
            ]
            response = model.chat.completions.create(
                model='gpt-4-0613',
                messages=message,
                temperature=0
            )
            
            # add to output
            output.append({
                "product 1": left,
                "product 2": right,
                "label": label,
                "response": response.choices[0].message.content
            })
            count += 1
        
        with open(f"./with-schema-gpt4/{name}_results.json", "w") as out:
            json.dump(output, out, indent=4)